In [1]:
import random
import sys

try:
    sys.path.remove('/Users/vblot/Documents/projects/mapie/original/MAPIE')
    sys.path.remove('/Users/vblot/Documents/projects/mapie/original/MAPIE')
except:
    pass
sys.path.append("/Users/vblot/Documents/projects/cemracs/conformalized_gp/")
# sys.path.append("/Users/vblot/Documents/projects/cemracs/conformal_uq")

import matplotlib.pyplot as plt
import scipy
import numpy as np
import pandas as pd
from scipy.stats import bootstrap

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import mmd
from mapie.conformity_scores.residual_conformity_scores import GPCrossConformityScore
from mapie.metrics import regression_coverage_score_v2, regression_mean_width_score
from mapie.regression import MapieRegressor
from notebooks_gp.adaptivity_measures import hsic_ot, my_correlation_pearson, q2, my_correlation_spearman
from notebooks_gp.datasets import get_morokoff
from wrappers import GpOTtoSklearnStd


plt.rcParams["figure.dpi"] = 100
%load_ext autoreload
%autoreload 2


# Get data

In [2]:
DATA_PATH = "/Users/vblot/Documents/projects/cemracs/data_UQ"
EDF_BLUE = np.array([[26, 54, 105]]) / 255
CAP_BLUE = np.array([[48, 112, 170]]) / 255
PS_VIOLET = np.array([[84, 13, 46]]) / 255
EDF_ORANGE = np.array([[223, 84, 49]]) / 255
LISN_YELLOW = np.array([[242, 188, 64]]) / 255
METHOD = "plus"
N_POINTS_TRAIN = "all"
CV = 10
SAVE_PLOT = False


In [3]:
X, y = get_morokoff(noisy=True, nobs=200)

In [4]:
print(X.shape, y.shape)

(200, 10) (200,)


# Split into train test

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
n_train = N_POINTS_TRAIN
n_train = len(X_train) if type(n_train) == str else n_train
random.seed(42)
indexes_train = np.random.choice(range(len(X_train)), n_train, replace=False)
X_train = X_train[indexes_train]
y_train = y_train[indexes_train]

In [6]:
print(
    f"  N training points = {len(X_train)}\n",
    f" N testing points = {len(X_test)}"
)

  N training points = 160
  N testing points = 40


# Get data

In [7]:
scaler = StandardScaler().fit(X_train)
X_train_scale = scaler.transform(X_train)
X_test_scale = scaler.transform(X_test)

# Define all possible models

In [8]:
noise = .1

In [9]:
models_hp = {
    "GP": {
        "nu": {
            1/2: [1],
            3/2: [1],
            5/2: [1],
        }
    },
    "J+": {
        "nu": {
            1/2: [1],
            3/2: [1],
            5/2: [1],
        }
    },
    "J+GP": {
        "nu": {
            1/2: [.5, 1, 1.5],
            3/2: [.5, 1, 1.5],
            5/2: [.5, 1, 1.5],
        }
    },
    "J-minmax-GP": {
        "nu": {
            1/2: [.5, 1, 1.5],
            3/2: [.5, 1, 1.5],
            5/2: [.5, 1, 1.5],
        }
    }
}

In [10]:
models = {}

for method in models_hp.keys():
    for nu in models_hp[method]["nu"].keys():
        for power_std in models_hp[method]["nu"][nu]:
            models[(method, nu, power_std)] = {
                "estimator": GpOTtoSklearnStd(
                    scale=1, amplitude=1.0, nu=nu, noise=noise, power_std=power_std
                )
            }
            

In [11]:
models.keys()

dict_keys([('GP', 0.5, 1), ('GP', 1.5, 1), ('GP', 2.5, 1), ('J+', 0.5, 1), ('J+', 1.5, 1), ('J+', 2.5, 1), ('J+GP', 0.5, 0.5), ('J+GP', 0.5, 1), ('J+GP', 0.5, 1.5), ('J+GP', 1.5, 0.5), ('J+GP', 1.5, 1), ('J+GP', 1.5, 1.5), ('J+GP', 2.5, 0.5), ('J+GP', 2.5, 1), ('J+GP', 2.5, 1.5), ('J-minmax-GP', 0.5, 0.5), ('J-minmax-GP', 0.5, 1), ('J-minmax-GP', 0.5, 1.5), ('J-minmax-GP', 1.5, 0.5), ('J-minmax-GP', 1.5, 1), ('J-minmax-GP', 1.5, 1.5), ('J-minmax-GP', 2.5, 0.5), ('J-minmax-GP', 2.5, 1), ('J-minmax-GP', 2.5, 1.5)])

In [12]:
for model_name, model in models.items():
    if model_name[0] == "GP":
        print("Fitting ", model_name)
        model["estimator"].fit(X_train_scale, y_train)

Fitting  ('GP', 0.5, 1)
Fitting  ('GP', 1.5, 1)
Fitting  ('GP', 2.5, 1)


WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.78326,7.57666,6.78734,6.92692,6.5869]#10


In [13]:
for model_name, model in models.items():
    if model_name[0] == "GP":
        print(model_name, "MSE:", mean_squared_error(y_test, model["estimator"].predict(X_test_scale)))

('GP', 0.5, 1) MSE: 0.0028370670805225365
('GP', 1.5, 1) MSE: 0.0026025693688943663
('GP', 2.5, 1) MSE: 0.002517261748309972


In [14]:
for model_name, model in models.items():
    if model_name[0] == "GP":
        print(model_name, "Q2:", q2(y_test, model["estimator"].predict(X_test_scale)))

('GP', 0.5, 1) Q2: 0.8275736061228423
('GP', 1.5, 1) Q2: 0.8418255055812942
('GP', 2.5, 1) Q2: 0.8470101857351731


# Fit MAPIE

In [15]:
for model_name, model in models.items():
    if model_name[0] != "GP":
        if "+" in model_name[0]:
            METHOD = "plus"
        else:
            METHOD = "minmax"
        models[model_name]["mapie_estimator"] = MapieRegressor(
            estimator=model["estimator"],
            conformity_score=GPCrossConformityScore(sym=True) if "GP" in model_name[0] else None,
            cv=CV,
            method=METHOD,
            model_has_std=True if "GP" in model_name[0] else False,
            random_state=42
        )


In [16]:
for model_name, model in models.items():
    if model_name[0] != "GP":
        print("Fitting MAPIE", model_name)
        model["mapie_estimator"].fit(X_train_scale, y_train)


Fitting MAPIE ('J+', 0.5, 1)


WRN - (previous message repeated 5 times)
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.6169,6.78326,7.57666,6.78734,6.84665,6.48673]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.77364,6.66584,6.73579,6.78326,7.57666,6.77061,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.00126,6.95358,6.81422,6.59889,6.73579,6.78326,7.54532,6.71692,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.70357,7.57666,6.78734,6.8971,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.78326,7.45123,6.78734,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.78326,7.57666,6.78734,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04365,6.91888,6.81422,6.67021,6.68359,6.78326,7.57666

Fitting MAPIE ('J+', 1.5, 1)


WRN - (previous message repeated 3 times)
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.6169,6.78326,7.57666,6.78734,6.84665,6.48673]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.77364,6.66584,6.73579,6.78326,7.57666,6.77061,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.00126,6.95358,6.81422,6.59889,6.73579,6.78326,7.54532,6.71692,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.70357,7.57666,6.78734,6.8971,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.78326,7.45123,6.78734,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.78326,7.57666,6.78734,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04365,6.91888,6.81422,6.67021,6.68359,6.78326,7.57666

Fitting MAPIE ('J+', 2.5, 1)


...

WRN - (previous message repeated 3 times)
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.6169,6.78326,7.57666,6.78734,6.84665,6.48673]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.77364,6.66584,6.73579,6.78326,7.57666,6.77061,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.00126,6.95358,6.81422,6.59889,6.73579,6.78326,7.54532,6.71692,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.70357,7.57666,6.78734,6.8971,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.78326,7.45123,6.78734,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.78326,7.57666,6.78734,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04365,6.91888,6.81422,6.67021,6.68359,6.78326,7.57666

Fitting MAPIE ('J-minmax-GP', 2.5, 1)


WRN - (previous message repeated 3 times)
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.6169,6.78326,7.57666,6.78734,6.84665,6.48673]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.77364,6.66584,6.73579,6.78326,7.57666,6.77061,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.00126,6.95358,6.81422,6.59889,6.73579,6.78326,7.54532,6.71692,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.70357,7.57666,6.78734,6.8971,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.78326,7.45123,6.78734,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.78326,7.57666,6.78734,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04365,6.91888,6.81422,6.67021,6.68359,6.78326,7.57666

Fitting MAPIE ('J-minmax-GP', 2.5, 1.5)


WRN - (previous message repeated 3 times)
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.6169,6.78326,7.57666,6.78734,6.84665,6.48673]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.77364,6.66584,6.73579,6.78326,7.57666,6.77061,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.00126,6.95358,6.81422,6.59889,6.73579,6.78326,7.54532,6.71692,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.70357,7.57666,6.78734,6.8971,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.78326,7.45123,6.78734,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04392,6.99565,6.81422,6.67021,6.73579,6.78326,7.57666,6.78734,6.92692,6.5869]#10
WRN - Warning! For coherency we set scale upper bounds = [7.04365,6.91888,6.81422,6.67021,6.68359,6.78326,7.57666

# Coverage

In [17]:
ALPHA = np.array([.1, .05, .01])
q_alpha_min = scipy.stats.norm.ppf(ALPHA / 2)
q_alpha_max = scipy.stats.norm.ppf(1 - ALPHA / 2)
for model_name, model in models.items():
    if model_name[0] == "GP":
        y_mean, y_std = model["estimator"].predict(X_test_scale, return_std=True)
        y_pss_gp = np.concatenate(
            [
                (y_mean.reshape(-1, 1) + y_std.reshape(-1, 1) * q_alpha_min.reshape(1, -1))[:, np.newaxis, :],
                (y_mean.reshape(-1, 1) + y_std.reshape(-1, 1) * q_alpha_max.reshape(1, -1))[:, np.newaxis, :]
            ],
            axis=1
        )
        model["y_pss"] = y_pss_gp

In [18]:
for model_name, model in models.items():
    if model_name[0] != "GP":
        print("Predict MAPIE", model_name)
        _, y_pss = model["mapie_estimator"].predict(X_test_scale, alpha=ALPHA)
        model["y_pss"] = y_pss


Predict MAPIE ('J+', 0.5, 1)
Predict MAPIE ('J+', 1.5, 1)
Predict MAPIE ('J+', 2.5, 1)
Predict MAPIE ('J+GP', 0.5, 0.5)
Predict MAPIE ('J+GP', 0.5, 1)
Predict MAPIE ('J+GP', 0.5, 1.5)
Predict MAPIE ('J+GP', 1.5, 0.5)
Predict MAPIE ('J+GP', 1.5, 1)
Predict MAPIE ('J+GP', 1.5, 1.5)
Predict MAPIE ('J+GP', 2.5, 0.5)
Predict MAPIE ('J+GP', 2.5, 1)
Predict MAPIE ('J+GP', 2.5, 1.5)
Predict MAPIE ('J-minmax-GP', 0.5, 0.5)
Predict MAPIE ('J-minmax-GP', 0.5, 1)
Predict MAPIE ('J-minmax-GP', 0.5, 1.5)
Predict MAPIE ('J-minmax-GP', 1.5, 0.5)
Predict MAPIE ('J-minmax-GP', 1.5, 1)
Predict MAPIE ('J-minmax-GP', 1.5, 1.5)
Predict MAPIE ('J-minmax-GP', 2.5, 0.5)
Predict MAPIE ('J-minmax-GP', 2.5, 1)
Predict MAPIE ('J-minmax-GP', 2.5, 1.5)


In [19]:
for model_name, model in models.items():
    model["coverage"] = [
        regression_coverage_score_v2(y_test, model["y_pss"][:, :, i])
        for i, _ in enumerate(ALPHA)
    ]
    model["average_width"] = [
        regression_mean_width_score(
            model["y_pss"][:, 1, i], model["y_pss"][:, 0, i]
        )
        for i, _ in enumerate(ALPHA)
    ]
    model["median_width"] = [
        np.median(np.abs(
             model["y_pss"][:, 1, i] -  model["y_pss"][:, 0, i]
        ))
        for i, _ in enumerate(ALPHA)
    ]
    model["var_width"] = [
        np.std(np.abs(
             model["y_pss"][:, 1, i] -  model["y_pss"][:, 0, i]
        ))
        for i, _ in enumerate(ALPHA)
    ]


# Correlation between width of the Prediction Interval and the error of the model


In [20]:
for model_name, model in models.items():
    if model_name[0] != "GP":
        model["errors"] = np.abs(model["mapie_estimator"].predict(X_test_scale, alpha=None) - y_test)
        model["width"] = np.abs(model["y_pss"][:, 0, :] - model["y_pss"][:, 1, :])
    else:
        model["errors"] = np.abs(model["estimator"].predict(X_test_scale) - y_test)
        model["width"] = np.abs(model["y_pss"][:, 0, :] - model["y_pss"][:, 1, :])


In [21]:
model["width"].shape

(40, 3)

In [22]:
for model_name, model in models.items():
    model["pearson_correlation_to_error"] = []
    model["spearman_correlation_to_error"] = []
    model["hsic_to_error"] = []
    for index_confidence in range(len(ALPHA)):
        str_vect = [
            e + '--' + w for e, w in zip(
                model["errors"].astype("str").tolist(),
                model["width"][:, index_confidence].astype("str").tolist()
            )
        ]
        model["pearson_correlation_to_error"].append(bootstrap(
            (np.array(str_vect), ),
            my_correlation_pearson,
            axis=0
        ))
        model["spearman_correlation_to_error"].append(bootstrap(
            (np.array(str_vect), ),
            my_correlation_spearman,
            axis=0
        ))
        model["hsic_to_error"].append(hsic_ot(
            model["errors"],
            model["width"][:, index_confidence]
        ))


/Users/vblot/anaconda3/envs/cemracs/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4461: NearConstantInputWarning: An input array is nearly constant; the computed correlation coefficient may be inaccurate.
  warnings.warn(stats.NearConstantInputWarning(msg))


# Size of the Prediction Intervals

In [23]:
# Get average width of the prediction intervals for each model in a pandas DataFrame
index = pd.MultiIndex.from_tuples([("Average width", i) for i in 1 - ALPHA])
df_width = pd.DataFrame(
    {
        model_name: model["average_width"]
        for model_name, model in models.items()
    },
    index=index
)


In [24]:
# Get average std of the prediction intervals for each model in a pandas DataFrame
index = pd.MultiIndex.from_tuples([("Std width", i) for i in 1 - ALPHA])

df_std = pd.DataFrame(
    {
        model_name: model["var_width"]
        for model_name, model in models.items()
    },
    index=index
)


In [25]:
index = pd.MultiIndex.from_tuples([("Coverage", i) for i in 1 - ALPHA])

df_cov = pd.DataFrame(
    {
        model_name: [c[0] for c in model["coverage"]]
        for model_name, model in models.items()
    },
    index=index
)


In [26]:
index = pd.MultiIndex.from_tuples([("Spearman Correlation", i) for i in 1 - ALPHA])

df_spearman = pd.DataFrame(
    {
        model_name: [np.mean(c.bootstrap_distribution) for c in model["spearman_correlation_to_error"]]
        for model_name, model in models.items()
    },
    index=index
)


In [27]:
index = pd.MultiIndex.from_tuples([("Pearson Correlation", i) for i in 1 - ALPHA])

df_pearson= pd.DataFrame(
    {
        model_name: [np.mean(c.bootstrap_distribution) for c in model["pearson_correlation_to_error"]]
        for model_name, model in models.items()
    },
    index=index
)

In [28]:
pd.concat([df_width.T, df_std.T, df_cov.T, df_spearman.T, df_pearson.T], axis=1)

Average width                     Std width            \
                             0.90      0.95      0.99      0.90      0.95   
GP          0.5 1.0      0.199038  0.237169  0.311693  0.014167  0.016882   
            1.5 1.0      0.123542  0.147210  0.193466  0.019885  0.023695   
            2.5 1.0      0.106982  0.127477  0.167533  0.021101  0.025143   
J+          0.5 1.0      0.183613  0.263643  0.407983  0.004125  0.003403   
            1.5 1.0      0.165734  0.228001  0.354142  0.004209  0.004619   
            2.5 1.0      0.164689  0.220060  0.333407  0.003581  0.004908   
J+GP        0.5 0.5      0.176242  0.252329  0.374118  0.007920  0.009085   
                1.0      0.168474  0.241124  0.344706  0.013394  0.017077   
                1.5      0.162475  0.232329  0.324409  0.018524  0.024391   
            1.5 0.5      0.155187  0.207831  0.310684  0.013886  0.018342   
                1.0      0.149021  0.190105  0.278980  0.025115  0.031373   
                1.5      0.145507  0.176324  0.248467  0.035400  0.041992   
            2.5 0.5      0.153509  0.195998  0.297096  0.015990  0.020642   
                1.0      0.144820  0.178866  0.254358  0.029307  0.035009   
                1.5      0.141946  0.169949  0.223156  0.042260  0.048474   
J-minmax-GP 0.5 0.5      0.189019  0.277713  0.396326  0.011934  0.014485   
                1.0      0.182514  0.265526  0.365648  0.016770  0.021765   
                1.5      0.178304  0.258263  0.338364  0.021684  0.029228   
            1.5 0.5      0.179446  0.231749  0.336544  0.018396  0.021784   
                1.0      0.170186  0.214113  0.298633  0.028166  0.034866   
                1.5      0.167956  0.198205  0.266171  0.039424  0.046048   
            2.5 0.5      0.176460  0.220865  0.327090  0.021471  0.024730   
                1.0      0.169247  0.201927  0.272178  0.033696  0.039682   
                1.5      0.168195  0.192134  0.235868  0.047136  0.052535   

                              Coverage               Spearman Correlation  \
                         0.99     0.90   0.95   0.99                 0.90   
GP          0.5 1.0  0.022186    0.925  0.975  1.000             0.323449   
            1.5 1.0  0.031140    0.850  0.875  0.950             0.267510   
            2.5 1.0  0.033044    0.750  0.875  0.950             0.252056   
J+          0.5 1.0  0.001947    0.900  0.950  1.000             0.161933   
            1.5 1.0  0.005679    0.925  0.975  1.000             0.073455   
            2.5 1.0  0.003696    0.900  0.950  1.000             0.074026   
J+GP        0.5 0.5  0.013140    0.900  0.975  1.000             0.397584   
                1.0  0.024940    0.900  0.975  1.000             0.373616   
                1.5  0.036632    0.900  0.975  1.000             0.379562   
            1.5 0.5  0.024296    0.875  0.950  1.000             0.311348   
                1.0  0.047769    0.875  0.950  1.000             0.246928   
                1.5  0.061872    0.900  0.950  1.000             0.262796   
            2.5 0.5  0.030301    0.875  0.950  1.000             0.237161   
                1.0  0.054480    0.900  0.950  1.000             0.237802   
                1.5  0.066744    0.900  0.950  0.975             0.245616   
J-minmax-GP 0.5 0.5  0.018649    0.925  1.000  1.000             0.375202   
                1.0  0.029564    0.925  1.000  1.000             0.395672   
                1.5  0.038671    0.925  1.000  1.000             0.367470   
            1.5 0.5  0.029981    0.925  0.975  1.000             0.328651   
                1.0  0.047912    0.925  0.950  1.000             0.364356   
                1.5  0.061907    0.950  0.950  1.000             0.351195   
            2.5 0.5  0.036199    0.925  0.950  1.000             0.319267   
                1.0  0.053909    0.950  0.950  1.000             0.341157   
                1.5  0.066373    0.950  0.950  1.000             0.330960   

              